## Homework 3 ASR task
This is all my code for HW3. You can find all (surviving, unfortunately wandb got full and we deleted all runs by mistake) runs in the wandb link [here](https://wandb.ai/abk171-university-of-pittsburgh/hw3p2-ablations?nw=nwuserabk171). \\
My ablations were in 3 main areas:

1. Embedding: \\
    I tried few simple convolution networks but eventually settled on convnext (partly because I couldnt get a chance to implement it in hw2). I use a `conv1d` layer with stride 2 and kernel size 15 as this was the best context window size in HW1P2 for me.

    The best embedding layer is like so:
    
    ```
    self.embedding = torch.nn.Sequential(
        nn.Conv1d(in_channels= input_size,
                out_channels= 48,
                kernel_size= config['kernel_size'],
                stride= 2,
                padding= 1),
        ConvNeXtBlock1D(dim=48, out_dim=96),
        ConvNeXtBlock1D(dim=96, out_dim=192),
        ConvNeXtBlock1D(dim=192, out_dim=192),
        ConvNeXtBlock1D(dim=192, out_dim=192),
        ConvNeXtBlock1D(dim=192, out_dim=384)
    )
    ```
    This is a very small implementation of convnext which follows the recommended 1:1:3:1 ratio and cuts the dimension sizes by half.

2. PBLSTM: \\
    Simple pBLSTM, only used 1 layer.

3. Decoder: \\
    Again, related to the best archtecture from my HW1P2.
    However, I changed to `GELU()` activation. I experimented with `dropout` and `batchnorm` but wasn't able to see good results. I suspect because of very low learning rate.


Further, I also implemented time and frequency masking, but hardly saw any improvement from those.

I kept the beam widths as high as I could in both cases to try and get stable results.

Thank you for taking the time to grade my work.

# Installs

In [ ]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 769.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 124.9 MB/s eta 0:00:00



This may take a while

In [ ]:
!pip install torchsummaryX==1.3.0
!pip install pandas==1.5.2
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.17.0 requires pandas>=1.5.3, but you have pandas 1.5.2 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 1.5.2 which is incompatible.
mizani 0.11.4 requires pandas>=2.1.0, but you have pandas 1.5.2 which is incompatible.
plotnine 0.13.6 requires pandas<3.0.0,>=2.1.0, but you have pandas 1.5.2 which is incompatible.
xarray 2024.9.0 requires pandas>=2.1, but you have pandas 1.5.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"abhigyankishor","key":"ce53f67a6b84738c6b11d7917e02e653"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!kaggle competitions download -c 11-785-hw3p2-f24

 99% 3.95G/3.97G [00:26<00:00, 188MB/s]
100% 3.97G/3.97G [00:26<00:00, 159MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f24-hw3p2  ctcdecode  hw3p2asr-f24.zip  sample_data
'''
!unzip -q 11-785-hw3p2-f24.zip
!ls

11785-f24-hw3p2  11-785-hw3p2-f24.zip  ctcdecode  sample_data


# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]



In [ ]:
# You might want to play around with the mapping as a sanity check here


### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self,
                 PHONEMES,
                 mfcc_dir,
                 transcript_dir,
                 freq_mask_param,
                 time_mask_param,
                 fraction= 1,):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them
        self.time_mask = tat.TimeMasking(time_mask_param= time_mask_param, iid_masks=True)
        self.freq_mask = tat.FrequencyMasking(freq_mask_param= freq_mask_param, iid_masks=True)

        self.mfcc_dir = mfcc_dir
        self.transcript_dir = transcript_dir

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.phonemes = PHONEMES
        self.fraction = fraction

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS
        self.phoneme_map = {p: i for i, p in enumerate(self.phonemes)}

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        self.mfccs, self.transcripts = [], []

        for i, (mfcc_file, transcript_file) in enumerate(zip(self.mfcc_files, self.transcript_files)):
            if i == int(self.length * fraction):
                break
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_file))
            transcript = np.load(os.path.join(self.transcript_dir, transcript_file))[1:-1]

            mean_mfcc   = np.mean(mfcc, axis = 0)
            std_mfcc = np.std(mfcc, axis = 0) + 1e-8
            mfcc  = (mfcc - mean_mfcc) / std_mfcc

            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return int(self.length * self.fraction)

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        # raise NotImplemented

        mfcc =  self.mfccs[ind]# TODO
        transcript = self.transcripts[ind]# TODO
        mapped_transcript = np.vectorize(lambda x: self.phoneme_map[x])(transcript)
        return torch.FloatTensor(mfcc), torch.tensor(mapped_transcript)


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        batch_mfcc, batch_transcript = zip(*batch)

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = torch.tensor([mfcc.shape[0] for mfcc in batch_mfcc]) # TODO

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True) # TODO
        lengths_transcript = torch.tensor([transcript.shape[0] for transcript in batch_transcript]) # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        # batch_mfcc_pad.transpose(1, 2)
        # batch_mfcc_pad = self.time_mask(batch_mfcc_pad)
        # batch_mfcc_pad = self.freq_mask(batch_mfcc_pad)
        # batch_mfcc_pad.transpose(1, 2)

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

In [ ]:
class AudioDatasetTest(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self,
                 PHONEMES,
                 mfcc_dir,
                 fraction= 1):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = mfcc_dir

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

        self.phonemes = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        self.fraction = fraction

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS
        self.phoneme_map = {p: i for i, p in enumerate(self.phonemes)}

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        self.mfccs = []

        for i, mfcc_file in enumerate(self.mfcc_files):
            if i == int(self.length * fraction):
                break
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_file))

            mean_mfcc   = np.mean(mfcc, axis = 0)
            std_mfcc = np.std(mfcc, axis = 0) + 1e-8
            mfcc  = (mfcc - mean_mfcc) / std_mfcc

            self.mfccs.append(mfcc)
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return int(self.length * self.fraction)

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        # raise NotImplemented

        mfcc =  self.mfccs[ind]# TODO
        return torch.FloatTensor(mfcc)



    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        batch_mfcc= [mfcc for mfcc in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = torch.tensor([mfcc.shape[0] for mfcc in batch_mfcc]) # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?


        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)



### Config - Hyperparameters

In [ ]:
root = '/content/11-785-f24-hw3p2/'

config = {
    ## data directories
    'mfcc_train_dir'         : '/content/11785-f24-hw3p2/train-clean-100/mfcc',
    'mfcc_val_dir'           : '/content/11785-f24-hw3p2/dev-clean/mfcc',
    'mfcc_test_dir'          : '/content/11785-f24-hw3p2/test-clean/mfcc',

    'transcript_train_dir'   : '/content/11785-f24-hw3p2/train-clean-100/transcript',
    'transcript_val_dir'     : '/content/11785-f24-hw3p2/dev-clean/transcript',

    'time_mask_param'        : 0,
    'freq_mask_param'        : 0,

    'fraction'               : 1,



    # general
    "beam_width" : 4,
    "lr"         : 2e-3,
    "epochs"     : 50,
    "batch_size" : 64,  # Increase if your device can handle it

    # basic network
    "embedding_size" : 64,
    "hidden_size"    : 256,
    "kernel_size"    : 15,
    "dropout"        : 0.2,
    "bidirectional"  : True,

    #optimizer
    'weight_decay'   : 1e-4,

    #scheduler
    'factor'         : 0.8,
    'patience'       : 2,
    'threshold'      : 1e-1,
    'min_lr'         : 1e-5
}

# Feel free to add more items here

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [ ]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(PHONEMES, config['mfcc_train_dir'], config['transcript_train_dir'], config['freq_mask_param'], config['time_mask_param'], config['fraction']) #TODO
val_data = AudioDataset(PHONEMES, config['mfcc_val_dir'], config['transcript_val_dir'], 0, 0, config['fraction']) # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(PHONEMES, config['mfcc_test_dir'], config['fraction']) #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
    collate_fn  = train_data.collate_fn
)
val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 1,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = val_data.collate_fn
)
test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 1,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = test_data.collate_fn
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1696, 28]) torch.Size([64, 199]) torch.Size([64]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        self.embedding = nn.Conv1d(in_channels= 28,
                                         out_channels= config['embedding_size'],
                                         kernel_size= config['kernel_size'],
                                         stride= 2,
                                         padding= 1)


        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = config['embedding_size'],
                            hidden_size = config['hidden_size'],
                            num_layers = 3,
                            batch_first= True,
                            dropout= config['dropout'],
                            bidirectional= config['bidirectional'])

        recv_size = 2* config['batch_size'] if config['bidirectional'] else config['batch_size']
        self.classification = torch.nn.Sequential( #maybe change the hidden size to be normal 256,512...
            # PermuteBlock(),
            # torch.nn.BatchNorm1d(2 * config['hidden_size'] if config['bidirectional'] else config['hidden_size']),
            # PermuteBlock(),
            torch.nn.Linear(in_features=recv_size,out_features=len(PHONEMES)),
            # torch.nn.GELU(),
            # torch.nn.Linear(in_features=128, out_features=64),
            # torch.nn.GELU(),
            # torch.nn.Linear(in_features=64, out_features=len(PHONEMES))
        )


        self.logSoftmax = nn.LogSoftmax(dim=-1)#TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints

        x = x.transpose(1, 2).contiguous()
        x = self.embedding(x)
        x = x.transpose(1, 2).contiguous()

        lx = (lx + 2 - config['kernel_size']) // 2  + 1

        x = pack_padded_sequence(x, lx, batch_first= True, enforce_sorted= False)
        x = self.lstm(x)[0]
        x, lx = pad_packed_sequence(x, batch_first= True)

        x = self.classification(x)
        x = self.logSoftmax(x)

        return x, lx


## Initialize Basic Network
(If trying out the basic Network)

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                                         Kernel Shape      Output Shape  \
Layer                                                                     
0_embedding.0.Linear_residual_connection     [28, 64]   [256, 2936, 64]   
1_embedding.0.Conv1d_dwconv               [1, 28, 15]   [256, 28, 2936]   
2_embedding.0.LayerNorm_norm                     [28]   [256, 2936, 28]   
3_embedding.0.Linear_pwconv1                [28, 112]  [256, 2936, 112]   
4_embedding.0.GELU_act                              -  [256, 2936, 112]   
5_embedding.0.Linear_pwconv2                [112, 64]   [256, 2936, 64]   
6_lstm                                              -      [89014, 512]   
7_classification.Linear_0                   [512, 41]   [256, 1462, 41]   
8_logSoftmax                                        -   [256, 1462, 41]   

                                             Params  Mult-Adds  
Layer                                                           
0_embedding.0.Linear_residual_connection    

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding.0.Linear_residual_connection,"[28, 64]","[256, 2936, 64]",1856.0,1792.0
1_embedding.0.Conv1d_dwconv,"[1, 28, 15]","[256, 28, 2936]",448.0,1233120.0
2_embedding.0.LayerNorm_norm,[28],"[256, 2936, 28]",56.0,28.0
3_embedding.0.Linear_pwconv1,"[28, 112]","[256, 2936, 112]",3248.0,3136.0
4_embedding.0.GELU_act,-,"[256, 2936, 112]",NaN,NaN
5_embedding.0.Linear_pwconv2,"[112, 64]","[256, 2936, 64]",7232.0,7168.0
6_lstm,-,"[89014, 512]",3813376.0,3801088.0
7_classification.Linear_0,"[512, 41]","[256, 1462, 41]",21033.0,20992.0
8_logSoftmax,-,"[256, 1462, 41]",NaN,NaN


## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class LockedDropout(nn.Module):
    def __init__(self, p = 0.25):
        super().__init__()
        self.p = p

    def forward(self, x):
        if not self.training or not self.p:
            return x
        x = x.clone()
        mask = x.new_empty(1, x.size(1), x.size(2), requires_grad=False).bernoulli_(1 - self.p)
        mask = mask.div_(1 - self.p)
        mask = mask.expand_as(x)
        return x * mask
    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'p=' + str(self.p) + ')'

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = torch.nn.LSTM(input_size=2*input_size, hidden_size=hidden_size, num_layers=3, bidirectional=True, dropout = 0.3)# TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence
        seq_unpacked, lens_unpacked = pad_packed_sequence(x_packed, batch_first=True)
        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        x, x_lens = self.trunc_reshape(x=seq_unpacked, x_lens=lens_unpacked)
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # TODO: Pack Padded Sequence. What output(s) would you get?
        PackedSequence = pack_padded_sequence(input=x,lengths=x_lens,batch_first=True,enforce_sorted=False)
        # TODO: Pass the sequence through bLSTM
        packed_output, (_, _) = self.blstm(PackedSequence)
        # What do you return?

        return packed_output

    def trunc_reshape(self, x, x_lens):
        b, t, c = x.shape
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        if t % 2 == 1:
            x = x[:, :-1, :]
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        x = x.reshape(b, x.shape[1] // 2, c * 2)
        # TODO: Reduce lengths by the same downsampling factor
        return x, x_lens / 2

### Encoder

In [ ]:
import torch
import torch.nn as nn

class ConvNeXtBlock1D(nn.Module):
    def __init__(self, dim, out_dim, layer_scale_init_value=1e-6, drop_path=0.0):
        super().__init__()

        self.dwconv = nn.Conv1d(dim, dim, kernel_size=7, padding=3, groups=dim)  # depthwise conv

        self.norm = nn.LayerNorm(dim)

        self.pwconv1 = nn.Linear(dim, 4 * dim)
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, out_dim)

        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((out_dim,)), requires_grad=True) if layer_scale_init_value > 0 else None

        self.residual_connection = nn.Linear(dim,out_dim) if dim != out_dim else nn.Identity()

    def forward(self, x):
        residual = x
        residual = residual.transpose(1, 2)
        residual = self.norm(residual)
        residual = self.residual_connection(residual)

        x = self.dwconv(x)

        x = x.transpose(1, 2)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)

        if self.gamma is not None:
          x = self.gamma * x

        x = x.transpose(1, 2)
        residual = residual.transpose(1,2)

        return residual + x

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()


       #TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.
        self.embedding = torch.nn.Sequential(
            nn.Conv1d(in_channels= input_size,
                    out_channels= 64,
                    kernel_size= config['kernel_size'],
                    stride= 2,
                    padding= 1),
            # ConvNeXtBlock1D(dim=input_size, out_dim=64),
            ConvNeXtBlock1D(dim=64, out_dim=128),
            ConvNeXtBlock1D(dim=128, out_dim=256),
            ConvNeXtBlock1D(dim=256, out_dim=256),
            ConvNeXtBlock1D(dim=256, out_dim=256),
            # ConvNeXtBlock1D(dim=256, out_dim=384)
        )

        # self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            # ...
            # ...

        # )
        self.lstm_layers = torch.nn.ModuleList()
        for i in range(1):
            input_size_pblstm = 256 if i == 0 else encoder_hidden_size * 2
            self.lstm_layers.append(pBLSTM(input_size=input_size_pblstm, hidden_size=encoder_hidden_size))
            self.lstm_layers.append(LockedDropout())

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        # TODO: Pad Packed Sequence

        x = x.transpose(1, 2).contiguous()
        x = self.embedding(x)
        x = x.transpose(1, 2).contiguous()

        x_lens = (x_lens + 2 - config['kernel_size']) // 2  + 1

        for layer in self.lstm_layers:
            if isinstance(layer, pBLSTM):
                x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
                x_packed = layer(x_packed)
                x, x_lens = pad_packed_sequence(x_packed, batch_first=True)
            else:
                x = torch.permute(x, (1, 0, 2))
                x = layer(x)
                x = torch.permute(x,(1,0,2))

        # Remember the number of output(s) each function returns

        return x, x_lens

### Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(embed_size), PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
             torch.nn.Linear(in_features=embed_size,out_features=128),
            torch.nn.GELU(),
            # torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=128, out_features=64),
            torch.nn.GELU(),
            # torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=64, out_features=output_size)
        )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification
        out = self.mlp(encoder_out)
        out = self.softmax(out)
        return out

## Initialize ASR Network

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()
        self.time_mask = tat.TimeMasking(time_mask_param= config['time_mask_param'], iid_masks=True)
        self.freq_mask = tat.FrequencyMasking(freq_mask_param= config['freq_mask_param'], iid_masks=True)
        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
            PermuteBlock(),
            self.time_mask,
            self.freq_mask,
            PermuteBlock()
        )
        self.encoder        = Encoder(
            input_size          = input_size,
            encoder_hidden_size = embed_size * 2
        )# TODO: Initialize Encoder
        self.decoder        = Decoder(
            embed_size          = 4 * embed_size,
            output_size         = output_size
        )# TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = ASRModel(
    input_size  = 28,#TODO,
    embed_size  = 192,#TODO
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device),lx)

ASRModel(
  (time_mask): TimeMasking()
  (freq_mask): FrequencyMasking()
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): TimeMasking()
    (2): FrequencyMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): Sequential(
      (0): Conv1d(28, 64, kernel_size=(15,), stride=(2,), padding=(1,))
      (1): ConvNeXtBlock1D(
        (dwconv): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(3,), groups=64)
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (pwconv1): Linear(in_features=64, out_features=256, bias=True)
        (act): GELU(approximate='none')
        (pwconv2): Linear(in_features=256, out_features=128, bias=True)
        (residual_connection): Linear(in_features=64, out_features=128, bias=True)
      )
      (2): ConvNeXtBlock1D(
        (dwconv): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,), groups=128)
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (pwconv1): L

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations.PermuteBlock_0,-,"[64, 28, 2936]",NaN,NaN
1_augmentations.TimeMasking_1,-,"[64, 28, 2936]",NaN,NaN
2_augmentations.TimeMasking_1,-,"[64, 28, 2936]",NaN,NaN
3_augmentations.FrequencyMasking_2,-,"[64, 28, 2936]",NaN,NaN
4_augmentations.FrequencyMasking_2,-,"[64, 28, 2936]",NaN,NaN
5_augmentations.PermuteBlock_3,-,"[64, 2936, 28]",NaN,NaN
6_encoder.embedding.Conv1d_0,"[28, 64, 15]","[64, 64, 1462]",26944.0,39298560.0
7_encoder.embedding.1.LayerNorm_norm,[64],"[64, 1462, 64]",128.0,64.0
8_encoder.embedding.1.Linear_residual_connection,"[64, 128]","[64, 1462, 128]",8320.0,8192.0


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [ ]:
#TODO


criterion = torch.nn.CTCLoss()# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(
    params= model.parameters(),
    lr= config['lr'],
    weight_decay= config['weight_decay']
) # What goes in here?

# optimizer = torch.optim.AdamW([
#     {'params': model.encoder.embedding.parameters(), 'lr': 3e-3},
#     {'params': model.encoder.lstm_layers.parameters(), 'lr': config['lr']},
#     {'params': model.decoder.parameters(), 'lr':3e-3}
# ], weight_decay= config['weight_decay'])

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(
    labels= LABELS,
    cutoff_top_n= 40,
    beam_width= config['beam_width'],
    log_probs_input= True,
    num_processes= 2,
)

#TODO

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer= optimizer,
                                                       patience=config['patience'],
                                                       threshold=config['threshold'],
                                                       factor=config['factor'], min_lr=config['min_lr'])#TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.

    beam_results, beam_scores, timesteps, out_lens = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings                    = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred = beam_results[i][0][:out_lens[i][0]]
        best_string = ''.join([PHONEME_MAP[x] for x in pred])
        pred_strings.append(best_string)
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]#TODO
        label_index = label[i, :label_lens[i]].tolist()
        label_string = ''.join([PHONEME_MAP[index] for index in label_index])
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    # raise NotImplemented
    return dist

# Test Implementation

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    # print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    # print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    # print(loss)
    # print(h.shape, lh.shape, y.shape, ly.shape)
    h = torch.permute(h, (1, 0, 2))
    print(calculate_levenshtein(h, y, lh, ly, decoder, LABELS))

    break

63.640625


# WandB

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="6e6412a1eff673cacdcda973f6e61422daaa9387") # API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abk171 (abk171-university-of-pittsburgh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "complete_better_embedding_lower_thresh", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = True, ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
valid_dist,█▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,0.002
train_loss,0.12979
valid_dist,6.13658
valid_loss,0.34818


In [ ]:
run.finish()

wandb: ERROR Control-C detected -- Run data was not synced


KeyboardInterrupt: 

# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    # metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = '/epoch_model.pth'#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = '/best_model.pth'#TODO set best model path

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])#TODO

    train_loss              = train_model(model=model,train_loader=train_loader,criterion=criterion,optimizer=optimizer)#TODO
    valid_loss, valid_dist  = validate_model(model=model,val_loader=val_loader,decoder=decoder,phoneme_map=LABELS)#TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    # wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        # wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/50


	Train Loss 1.5775	 Learning Rate 0.0020000
	Val Dist 15.2051%	 Val Loss 0.6909
Saved epoch model

Epoch: 2/50


	Train Loss 0.6019	 Learning Rate 0.0020000
	Val Dist 11.3153%	 Val Loss 0.5213
Saved epoch model

Epoch: 3/50


	Train Loss 0.4593	 Learning Rate 0.0020000
	Val Dist 9.6789%	 Val Loss 0.4510
Saved epoch model

Epoch: 4/50


	Train Loss 0.3813	 Learning Rate 0.0020000
	Val Dist 8.7610%	 Val Loss 0.4144
Saved epoch model

Epoch: 5/50


...

	Train Loss 0.0849	 Learning Rate 0.0008192
	Val Dist 5.7087%	 Val Loss 0.3629
Saved epoch model
Saved best model

Epoch: 20/50


	Train Loss 0.0807	 Learning Rate 0.0008192
	Val Dist 5.8486%	 Val Loss 0.3760
Saved epoch model

Epoch: 21/50


	Train Loss 0.0768	 Learning Rate 0.0008192
	Val Dist 5.7501%	 Val Loss 0.3731
Saved epoch model

Epoch: 22/50


Train:  39%|███▉      | 175/446 [00:54<01:24,  3.20it/s, loss=0.0707, lr=0.000819]

KeyboardInterrupt: 

# Generate Predictions and Submit to Kaggle

In [ ]:
model, _, _, _, _= load_model('/best_model.pth', model)

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 20#TODO

test_decoder    = CTCBeamDecoder(
    labels= LABELS,
    cutoff_top_n= 40,
    beam_width= TEST_BEAM_WIDTH,
    log_probs_input= True,
    num_processes= 2
)#TODO
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string= decode_prediction(h, lh, test_decoder, LABELS)# TODO call decode_prediction
    #TODO save the output in results array.
    for prediction in prediction_string:
        results.append(prediction)

    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing



100%|██████████| 41/41 [00:34<00:00,  1.18it/s]


In [ ]:
data_dir = "/content/11785-f24-hw3p2/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

In [ ]:
!kaggle competitions submit -c automatic-speech-recognition-asr-slack -f submission.csv -m "I made it!"


100% 209k/209k [00:00<00:00, 329kB/s]
Successfully submitted to Automatic Speech Recognition (ASR) Slack